In [1]:
import os
import json
import torch
torch.set_grad_enabled(False)
import numpy as np
from pprint import pprint
import sys
sys.path.append('../..')
from nebula.models.attention import TransformerEncoderModel, TransformerEncoderChunks
from nebula import PEDynamicFeatureExtractor
from nebula.preprocessing import JSONTokenizerNaive, JSONTokenizerBPE
from nebula.misc import get_path, clear_cuda_cache, set_random_seed
from nebula.constants import *
from bertviz import model_view, head_view
from collections import Counter

SCRIPT_PATH = get_path(type="notebook")
ROOT = os.path.join(SCRIPT_PATH, "..", "..")

DEVICE = "cpu"

In [2]:
# done three folds on that run
MODEL_IDX = 1

# BPE
datafolder = os.path.join(ROOT, r"evaluation\paper_sota\out_speakeasy\nebula_speakeasy_vocab_50000_seqlen_512")
model_folder = os.path.join(ROOT, r"evaluation\paper_sota\out_speakeasy\cv_nebula_limNone_r1763_t20\training_files")

# Whitespace
# datafolder = os.path.join(ROOT, r"evaluation\paper_ablation\out_tokenizer\nebula_whitespace_vocab_50000_seqlen_512")
# model_folder = os.path.join(ROOT, r"evaluation\paper_ablation\out_tokenizer\cv_whitespace_limNone_r1763_t5\training_files")

# LOADING OBJECTS
model_file = [x for x in os.listdir(model_folder) if x.endswith(".torch")][MODEL_IDX]
model_file_fullpath = os.path.join(model_folder, model_file)
state_dict = torch.load(model_file_fullpath)

with open(os.path.join(datafolder, f"tokenizer_50000_vocab.json")) as f:
    nebula_vocab = json.load(f)

# BPE
tokenizer = JSONTokenizerBPE(
    vocab_size=len(nebula_vocab),
    seq_len=512,
    model_path=os.path.join(ROOT, datafolder, r"tokenizer_50000.model")
)

# Whitespace
# tokenizer = JSONTokenizerNaive(
#     vocab_size=len(nebula_vocab),
#     seq_len=512,
#     vocab=nebula_vocab
# )

modelArch = {
        "vocab_size": len(nebula_vocab),
        "maxlen": 512,
        "chunk_size": 64,
        "dModel": 64,  # embedding & transformer dimension
        "nHeads": 8,  # number of heads in nn.MultiheadAttention
        "dHidden": 256,  # dimension of the feedforward network model in nn.TransformerEncoder
        "nLayers": 2,  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
        "numClasses": 1, # binary classification
        "hiddenNeurons": [64],
        "layerNorm": False,
        "dropout": 0.3,
        "mean_over_sequence": False,
        "norm_first": True
    }
model = TransformerEncoderChunks(**modelArch)
model.load_state_dict(state_dict)
_ = model.to(DEVICE)


In [84]:
def get_attn(x, model):
    attentions = []
    x = model.pos_encoder(model.encoder(x))
    for layer in model.transformer_encoder.layers:
        this_layer_attn = layer.self_attn(x,x,x, average_attn_weights=False)[1]
        attentions.append(this_layer_attn.cpu())
        x = layer(x)
    return attentions

def viz(start, end, attentions, exampleTokenized, threshold=0.005, layer=1, heads=[]):
    m = 1/threshold
    tokens = exampleTokenized[start:end]
    att_crop = [x[:, :, start:end, start:end]*m for x in attentions]
    head_view(att_crop, tokens, prettify_tokens=False, heads=heads, layer=layer)

def report_where_attends(seq_1, seq_2, token_location, tokenized_input):
    # TODO: somewhat broken?
    msg = ""
    token = tokenized_input[token_location]
    where_attends_seq_2 = np.where(seq_1 == token_location)[0]
    if len(where_attends_seq_2) > 0:
        counterpart_tokens = list(set([tokenized_input[x] for x in seq_2[where_attends_seq_2].tolist() if x < len(tokenized_input)]))
        if token in counterpart_tokens:
            counterpart_tokens.remove(token)
        if counterpart_tokens: # TODO: this part reports wrong results
            msg = f"\t  It has connections with tokens at following locations and values:\n"
            msg += f"\t\tLocated: {where_attends_seq_2}\n"
            msg += f"\t\tValues: {counterpart_tokens}\n"
    return msg

def analyze_attentions(attentions, tokenized_input, threshold=0.005, diff=20, most_common=5, types=["proximity", "frequency"], limit=20, verbose=True):
    counter = 0
    print(f"\n[!] Analyzing attentions based on {types}... ")
    for layer_nr, layer in enumerate(attentions):
        idxs = np.where(layer > threshold)
        print(f"\n[!] Total {len(idxs[0])} tokens has strong activations at layer {layer_nr} with threshold: {threshold}!")
        if len(idxs[0]) > 0:
            heads = idxs[1].tolist()
            attn_seq_1 = idxs[2]
            attn_seq_2 = idxs[3]

            if "proximity" in types:
                # PROXIMITY CHECK
                print(f"[*] Performing a token attention proximity check with token difference: {diff}... ")
                #for j, (a1, a2) in enumerate(zip(attn_seq_1, attn_seq_2)):
                # do the same for loop, but in reverse order
                for j, (a1, a2) in enumerate(zip(attn_seq_1, attn_seq_2)):
                
                    # check if difference between a1 and a2 is below threshold
                    if a1 > len(tokenized_input)-1 or a2 > len(tokenized_input)-1:
                        continue
                    token_at_a1 = tokenized_input[a1]
                    token_at_a2 = tokenized_input[a2]
                    if abs(a1 - a2) > diff:
                        if verbose:
                            print(f"\tThese two tokens has strong activations but are far (diff: {abs(a1 - a2)}):\n\t\t{token_at_a1}\n\t\t{token_at_a2}")
                    elif abs(a1 - a2) <= diff:
                        counter += 1
                        if counter > limit:
                            return
                        which_heads = heads[j]
                        print("\tThese two close sequence tokens has strong activations:", a1, a2, f"at layer {layer_nr} head", which_heads)
                        print(f"\t\tToken at location {a1} -> {token_at_a1}")
                        print(f"\t\tToken at location {a2} -> {token_at_a2}")
                        aa = sorted([a1, a2])
                        a1, a2 = aa[0]-5, aa[1]+5
                        if a1 < 0:
                            a1 = 0
                        if a2 > len(tokenized_input):
                            a2 = len(tokenized_input)
                        vizString = f"viz({a1}, {a2}, attentions, tokenized_input, layer={layer_nr}, heads=[{which_heads}])"
                        print("Calling:", vizString)
                        viz(a1, a2, attentions, tokenized_input, layer=layer_nr, heads=[which_heads])
            if "count" in types or "frequency" in types:
                # COUNTER CHECK
                print("[*] Performing a token frequency check ... ")
                c = attn_seq_2.tolist()
                c.extend((attn_seq_1.tolist()))
                c = Counter(c).most_common(most_common)        
                for token_location, token_frequency in c:
                    if token_location > len(tokenized_input):
                        continue
                    appear_in_heads = [heads[i] for i in np.where(attn_seq_1 == token_location)[0].tolist()]
                    appear_in_heads.extend([heads[i] for i in np.where(attn_seq_2 == token_location)[0].tolist()])
                    appear_in_heads = list(set(appear_in_heads))

                    token_value = tokenized_input[token_location]
                    msg1 = report_where_attends(attn_seq_1, attn_seq_2, token_location, tokenized_input)
                    msg2 = report_where_attends(attn_seq_2, attn_seq_2, token_location, tokenized_input)
                    if msg1 or msg2:
                        msg = f"\n\t> Token '{token_value}' location in sequence: {token_location}\n"
                        msg += f"\t  It appears {token_frequency} times, in layer {layer_nr}, heads: {appear_in_heads}\n"
                        msg += msg1 + msg2
                        print(msg)
                        
                        counter += 1
                        if counter > limit:
                            return

def get_attention_report(exampleFile, model, tokenizer, maxLen=512, random_seed=42, diff=20, threshold=0.004, limit=20, verbose=True):
    with open(exampleFile) as f:
        example = json.load(f)

    extractor = PEDynamicFeatureExtractor()
    exampleProcessed = extractor.filter_and_normalize_report(example[0])
    
    exampleTokenized = tokenizer.tokenize(exampleProcessed)
    if len(exampleTokenized) == 1:
        exampleTokenized = exampleTokenized[0]
    print(f" Length of tokenized input: {len(exampleTokenized)}")
    exampleEncoded = tokenizer.encode(exampleProcessed)
    x = torch.Tensor(exampleEncoded).long().reshape(1,-1).to(DEVICE)

    if x.shape[1] >= maxLen:
        x = x[:, :maxLen]
    elif x.shape[1] < maxLen:
        # x = np.pad(x, ((0,0), (0, maxLen - x.shape[1])), 'constant', constant_values=0)
        # but in torch
        x = torch.cat((x, torch.zeros((1, maxLen - x.shape[1])).long().to(DEVICE)), dim=1)
    logit = model(x).to(DEVICE)
    prob = torch.sigmoid(logit)
    print("[!] Model scores on this sample -- logit:", logit[0][0].numpy(), " | probability:", prob[0][0].numpy())

    clear_cuda_cache()
    set_random_seed(random_seed)
    attentions = get_attn(x, model)
    analyze_attentions(attentions, exampleTokenized, types=["proximity"], threshold=threshold, diff=diff, limit=limit, verbose=verbose)
    analyze_attentions(attentions, exampleTokenized, types=["frequency"], most_common=10, limit=limit, threshold=threshold)
    return attentions, exampleTokenized

In [85]:
exampleFile = os.path.join(ROOT, r"data\data_raw\windows_emulation_trainset\report_clean\0029db52469940e64c9755dd7b5794e889ea54dc9a59dd01d663f59ef4087523.json")
attentions, tokenized_input = get_attention_report(exampleFile, model, tokenizer, maxLen=512, random_seed=1492, diff=15, threshold=0.025, limit=10, verbose=False)

 Length of tokenized input: 3084
[!] Model scores on this sample -- logit: -15.306836  | probability: 2.2507406e-07

[!] Analyzing attentions based on ['proximity']... 

[!] Total 0 tokens has strong activations at layer 0 with threshold: 0.025!

[!] Total 23 tokens has strong activations at layer 1 with threshold: 0.025!
[*] Performing a token attention proximity check with token difference: 15... 
	These two close sequence tokens has strong activations: 202 209 at layer 1 head 7
		Token at location 202 -> args
		Token at location 209 -> ▁ap
Calling: viz(197, 214, attentions, tokenized_input, layer=1, heads=[7])


<IPython.core.display.Javascript object>

	These two close sequence tokens has strong activations: 223 209 at layer 1 head 7
		Token at location 223 -> ▁0x0
		Token at location 209 -> ▁ap
Calling: viz(204, 228, attentions, tokenized_input, layer=1, heads=[7])


<IPython.core.display.Javascript object>


[!] Analyzing attentions based on ['frequency']... 

[!] Total 0 tokens has strong activations at layer 0 with threshold: 0.025!

[!] Total 23 tokens has strong activations at layer 1 with threshold: 0.025!
[*] Performing a token frequency check ... 

	> Token 'ret' location in sequence: 60
	  It appears 1 times, in layer 1, heads: [0]
	  It has connections with tokens at following locations and values:
		Located: [0]
		Values: ['▁ap']


	> Token '▁0xfeee0001' location in sequence: 479
	  It appears 1 times, in layer 1, heads: [0]
	  It has connections with tokens at following locations and values:
		Located: [1]
		Values: ['▁ap']


	> Token 'args' location in sequence: 177
	  It appears 1 times, in layer 1, heads: [2]
	  It has connections with tokens at following locations and values:
		Located: [2]
		Values: ['▁ap']


	> Token 'system32' location in sequence: 153
	  It appears 1 times, in layer 1, heads: [6]
	  It has connections with tokens at following locations and values:
		Loc

# Strong activations

In [50]:
# timeout.exe -- msvcrt.memset
viz(148, 168, attentions, tokenized_input, layer=1, heads=[1])

<IPython.core.display.Javascript object>

In [60]:
# nslookup & ipconfig: start the same -> getsystemtimeasfiletime
viz(0, 21, attentions, tokenized_input, layer=1, heads=[2])
#viz(7, 21, attentions, tokenized_input, layer=1, heads=[2])

<IPython.core.display.Javascript object>